<a href="https://colab.research.google.com/github/Odaenethus/hello-github/blob/main/assignment_4_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U transformers datasets accelerate scipy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.9 MB/s eta 0:00:00


In [ ]:
import torch, numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", DEVICE)


Using: cuda


In [ ]:
ds = load_dataset("jingjietan/essays-big5")
print(ds["train"].column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/795k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/953k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1578 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/395 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/494 [00:00<?, ? examples/s]

['O', 'C', 'E', 'A', 'N', 'ptype', 'text', '__index_level_0__']


In [ ]:
ds = load_dataset("jingjietan/essays-big5") # Reload the dataset to its original state
TRAITS = ["O","C","E","A","N"]

def format_example(x):
    return {
        "text": x["text"],
        "labels": [float(x[t]) for t in TRAITS]
    }

ds = ds.map(format_example, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/1578 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

In [ ]:
MODEL = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize(batch):
    out = tokenizer(batch["text"], padding="max_length",
                    truncation=True, max_length=256)
    out["labels"] = batch["labels"]
    return out

ds = ds.map(tokenize, batched=True, remove_columns=["text"])
ds.set_format("torch")


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1578 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

def compute_metrics(pred):
    y_pred = pred.predictions
    y_true = pred.label_ids

    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred, multioutput="uniform_average")

    pearsons, spearmans = [], []
    for i in range(5):
        pearsons.append(pearsonr(y_true[:,i], y_pred[:,i])[0])
        spearmans.append(spearmanr(y_true[:,i], y_pred[:,i]).correlation)

    return {
        "mae": mae,
        "rmse": rmse,
        "r2": r2,
        "pearson": np.mean(pearsons),
        "spearman": np.mean(spearmans),
    }


In [ ]:
import torch.nn as nn
from transformers import AutoModel

class BigFiveRegressor(nn.Module):
    def __init__(self, freeze):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(MODEL)
        if freeze:
            for p in self.encoder.parameters():
                p.requires_grad = False
        self.head = nn.Linear(self.encoder.config.hidden_size, 5)
        self.loss = nn.MSELoss()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        x = self.encoder(input_ids=input_ids,
                         attention_mask=attention_mask).last_hidden_state[:,0]
        preds = self.head(x)
        loss = self.loss(preds, labels) if labels is not None else None
        return {"loss": loss, "logits": preds}


In [ ]:
from transformers import Trainer, TrainingArguments

def run(freeze, seed):
    torch.manual_seed(seed); np.random.seed(seed)

    model = BigFiveRegressor(freeze).to(DEVICE)

    args = TrainingArguments(
        output_dir="out",
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=5e-4 if freeze else 2e-5,
        per_device_train_batch_size=32 if freeze else 16,
        num_train_epochs=6 if freeze else 3,
        fp16=torch.cuda.is_available(),
        seed=seed,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds["train"],
        eval_dataset=ds["validation"],
        compute_metrics=compute_metrics
    )

    trainer.train()
    return trainer.evaluate(ds["test"])


In [ ]:
results = []
for s in [42,43,44]:
    r1 = run(freeze=True, seed=s)
    r1["approach"] = "frozen"
    r2 = run(freeze=False, seed=s)
    r2["approach"] = "fine_tune"
    results += [r1, r2]

import pandas as pd
pd.DataFrame(results)


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch,Training Loss,Validation Loss,Mae,Rmse,R2,Pearson,Spearman
1,No log,0.251671,0.494203,0.501668,-0.007810,0.074979,0.077830
2,No log,0.246286,0.488888,0.496272,0.013751,0.143579,0.141034
3,No log,0.245449,0.486683,0.495428,0.017103,0.159383,0.158832
4,No log,0.243831,0.484349,0.493793,0.023556,0.169461,0.168662
5,No log,0.244939,0.484415,0.494914,0.019145,0.170873,0.169769
6,No log,0.242732,0.483861,0.492679,0.027971,0.173709,0.172736


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


...

Epoch,Training Loss,Validation Loss,Mae,Rmse,R2,Pearson,Spearman
1,No log,0.254204,0.492648,0.504187,-0.017928,0.092514,0.089662
2,No log,0.245634,0.487360,0.495615,0.016363,0.146964,0.145551
3,No log,0.247345,0.485321,0.497338,0.009527,0.164728,0.165299
4,No log,0.242110,0.484763,0.492047,0.030473,0.173888,0.173178
5,No log,0.242067,0.483375,0.492003,0.030644,0.177046,0.177732
6,No log,0.242271,0.483098,0.492211,0.029825,0.178072,0.178260


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch,Training Loss,Validation Loss,Mae,Rmse,R2,Pearson,Spearman
1,No log,0.245907,0.488771,0.495890,0.015275,0.172352,0.168226
2,No log,0.241688,0.483455,0.491617,0.032169,0.184439,0.184014
3,No log,0.243024,0.478848,0.492974,0.026826,0.185124,0.186293


,eval_loss,eval_mae,eval_rmse,eval_r2,eval_pearson,eval_spearman,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch,approach
0,0.243679,0.484354,0.493638,0.023928,0.161156,0.158351,1.6888,292.509,36.712,6.0,frozen
1,0.242261,0.481338,0.492200,0.029604,0.169978,0.165492,0.9023,547.480,68.712,3.0,fine_tune
2,0.243675,0.484315,0.493635,0.023939,0.157025,0.156281,1.0311,479.103,60.130,6.0,frozen
3,0.242834,0.480032,0.492782,0.027307,0.171841,0.172045,0.9264,533.252,66.926,3.0,fine_tune
4,0.245300,0.486033,0.495278,0.017422,0.136850,0.135278,1.5175,325.537,40.857,6.0,frozen
5,0.244140,0.480079,0.494105,0.022073,0.161341,0.161927,0.9132,540.976,67.896,3.0,fine_tune
